In [1]:
import nltk
from nltk.corpus import stopwords
import re
import string
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix

In [2]:
stop = set(stopwords.words('english'))

text = "their are so many picture. how are you do this time very much!"
clean_text = []
for word in nltk.word_tokenize(text):
    if word not in stop:
        clean_text.append(word)
print(clean_text)

['many', 'picture', '.', 'time', 'much', '!']


In [3]:
example = "New competition launched :https://www.kaggle.com/c/nlp-getting-started"
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', text)


print(remove_URL(example))

New competition launched :


In [4]:
example = """<div>
<h1>Real or Fake</h1>
<p>Kaggle </p>
<a href="https://www.kaggle.com/c/nlp-getting-started">getting started</a>
</div>"""
def remove_html(text):
    html = re.compile(r'<.*?>')
    return html.sub(r'', text)

In [5]:
def remove_emoji(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [6]:
import string
def remove_punct(text):
    # 对punctuation中的词进行删除
    table = str.maketrans('', '', string.punctuation)
    return text.translate(table)

In [7]:
train_email = pd.read_csv(r"F:\BaiduNetdiskDownload\rb_mails\train.csv", usecols=[2], encoding='utf-8')
train_label = pd.read_csv(r"F:\BaiduNetdiskDownload\rb_mails\train.csv", usecols=[1], encoding='utf-8')
test_email = pd.read_csv(r"F:\BaiduNetdiskDownload\rb_mails\test_noLabel.csv", usecols=[1], encoding='utf-8')
# test_label = pd.read_csv(r"F:\BaiduNetdiskDownload\output_dtree.csv", usecols=[1], encoding='utf-8')

In [8]:
data_train = np.array(train_email).reshape((1, len(train_email)))[0].tolist()
label_train = np.array(train_label).reshape((1, len(train_email)))[0].tolist()

# data_dev = [data_train[i] for i in range(len(data_train)) if label_train[i] == "spam" ]
data_dev = np.array(test_email).reshape((1, len(test_email)))[0].tolist()
# label_dev = np.array(test_label).reshape((1, len(test_email)))[0].tolist()
data_dev[178]

'Yes see ya not on the dot'

In [9]:
def decision_tree(pred,item,data_dev):
    spam = r"F:\BaiduNetdiskDownload\rb_mails\spam_word.txt"
    ham = r"F:\BaiduNetdiskDownload\rb_mails\ham_word.txt"
    context = data_dev[item]
    #pd.read_csv(r"F:\BaiduNetdiskDownload\rb_mails\test_noLabel.csv", usecols=[1], encoding='utf-8')
    p = pred
    with open(spam, 'r') as f:
        spam_list = [line.strip() for line in f.readlines()]
    with open(ham, 'r') as f:
        ham_list = [line.strip() for line in f.readlines()]
    for string in spam_list:
        if string in context.lower():
            p = "spam"
    for string in ham_list:
        if string in context.lower():
            p = "ham"
    return p

In [11]:
# 使用词袋模型
vectorizer = CountVectorizer()
# CountVectorizer类会把文本全部转换为小写，然后将文本词块化。主要是分词，分标点
data_train_cnt = vectorizer.fit_transform(data_train)
data_test_cnt = vectorizer.transform(data_dev)

clf = MultinomialNB()
clf.fit(data_train_cnt, label_train)
# score = clf.score(data_test_cnt, label_dev)
k = clf.predict(data_test_cnt)
k_rate = clf.predict_proba(data_test_cnt)
cnt = 0
for item,pred in enumerate(k):
    # p = decision_tree(pred,item,data_dev)
    p = pred
    # test_label.loc[item] = p
    if k_rate[item][0]<0.9 and k_rate[item][0]>0.1:
        print(str(item+2) + ":" +data_dev[item] + p)
        cnt += 1
    # print(str(item) + ":" + p)
# test_label.to_csv(r'F:\BaiduNetdiskDownload\rb_mails\mlp\out.csv',index=False)
print(cnt)


29:Nokia phone is lovly..spam
32:Check Out Choose Your Babe Videos @ sms.shsex.netUN fgkslpoPW fgkslpoham
41:Mathews or tait or edwards or andersonspam
78:Hasn't that been the pattern recently crap weekends?ham
208:Sac needs to carry on:)ham
266:Can... I'm free...ham
280:Santa Calling! Would your little ones like a call from Santa Xmas eve? Call 09058094583 to book your time.spam
360:Download as many ringtones as u like no restrictions, 1000s 2 choose. U can even send 2 yr buddys. Txt Sir to 80082 ? spam
494:G.W.Rham
516:ringtoneking 84484spam
548:Ever thought about living a good life with a perfect partner? Just txt back NAME and AGE to join the mobile community. (100p/SMS)ham
568:FROM 88066 LOST ?2 HELPspam
651:I fetch yun or u fetch?ham
689:You will be receiving this week's Triple Echo ringtone shortly. Enjoy it!ham
700:Please CALL 08712402578 immediately as there is an urgent message waiting for youspam
707:As per your request 'Maangalyam (Alaipayuthe)' has been set as your callert

In [119]:
predt = clf.predict_proba(data_test_cnt)
print(predt[30])

[0.74727792 0.25272208]


In [120]:
data_dev[0]

'Ok.ok ok..then..whats ur todays plan'

In [40]:
spam = r"F:\BaiduNetdiskDownload\rb_mails\spam_word.txt"
ham = r"F:\BaiduNetdiskDownload\rb_mails\ham_word.txt"
context = "Hi babe its Chloe, how r u? I was smashed on saturday night, it was great! How was your weekend? U been missing me? SP visionsms.com Text stop to stop 150p/text"
pred = "first"
with open(spam, 'r') as f:
    spam_list = [line.strip() for line in f.readlines()]
with open(ham, 'r') as f:
    ham_list = [line.strip() for line in f.readlines()]
for string in spam_list:
    if string in context.lower():
        pred = "spam"
for string in ham_list:
    if string in context.lower():
        pred = "ham"
pred

'spam'

['fullonsms.com']

'spam'